In [1]:
from gensim import utils
from gensim.models.word2vec import Word2Vec
import pandas as pd
import numpy as np
import sklearn.datasets
import sklearn.linear_model

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df_train=pd.read_csv('train.tsv',sep='\t')

In [3]:
df_train=df_train[0:50000]

In [4]:
df_train=df_train.dropna()

In [5]:
df_train=df_train.join(pd.get_dummies(df_train['Sentiment']))

In [6]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment,0,1,2,3,4
0,1,1,A series of escapades demonstrating the adage ...,1,0,1,0,0,0
1,2,1,A series of escapades demonstrating the adage ...,2,0,0,1,0,0
2,3,1,A series,2,0,0,1,0,0
3,4,1,A,2,0,0,1,0,0
4,5,1,series,2,0,0,1,0,0


In [7]:
import nltk
import nltk.corpus
import nltk.stem.porter
porter_stemmer=nltk.stem.porter.PorterStemmer()
stopword=set(nltk.corpus.stopwords.words('english'))

In [8]:
bigl=[]
for string in df_train['Phrase']:
    lis=string.split()
    l=[]
    for word in lis:
        if word.isalpha()==True:
            #if word not in stopword:
            if len(word)!=1:
                word=word.lower()
                word=porter_stemmer.stem(word)
                l.append(word)
    bigl.append(l)

In [9]:
df_train['Feature Words']=bigl

In [10]:
df_train['Feature Words'][0:5]

0    [seri, of, escapad, demonstr, the, adag, that,...
1    [seri, of, escapad, demonstr, the, adag, that,...
2                                               [seri]
3                                                   []
4                                               [seri]
Name: Feature Words, dtype: object

In [11]:
num_features = 750  # Word vector dimensionality
min_word_count = 5 # Minimum word count
num_workers = 15    # Number of parallel threads
context = 5        # Context window size
downsampling = 1e-3 

In [12]:
model = Word2Vec(df_train['Feature Words'],\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

In [13]:
model.init_sims(replace=True)


In [14]:
model.wv.syn0.shape


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(5504, 750)

In [15]:
for words in df_train['Feature Words']:
    for word in words:
        if word in stopword:
            words.remove(word)

In [16]:
index2word_set = set(model.wv.index2word)

In [17]:
##Function to get average of all word vectors in a line

def av_vector_line(line):
    vector_feature=np.zeros(num_features,dtype='float32')
    num_words=0
    for word in line:
        if word in index2word_set:
            vector_feature=np.add(vector_feature,model[word])
            num_words+=1
    
    vector_feature=np.divide(vector_feature,num_words)
    return vector_feature

In [18]:
##Function to get the av feature matrix for training 

def av_feature_matrix(lines):
    av_feature=[]
    
    for line in lines:
        av_feature.append(av_vector_line(line))
    
    av_feature=np.array(av_feature)
    return av_feature

In [19]:
train_data_matrix=av_feature_matrix(df_train['Feature Words'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [22]:
y=df_train[[0,1,2,3,4]]

In [23]:
x=train_data_matrix
y=np.array(y)

In [21]:
z=np.isnan(x)

In [22]:
len(z)

50000

In [23]:
a=[]
for i in range(len(z)):
    if z[i].all()==True:
        a.append(i)

In [24]:
len(a)

1493

In [25]:
x=np.delete(x,a,axis=0)
y=np.delete(y,a)    

In [26]:
import sklearn.datasets
import sklearn.linear_model
x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(x,y,test_size=0.2)

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
rf = RandomForestClassifier(n_estimators=20,random_state=0)

In [29]:
rf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [30]:
rf.score(x_test,y_test)

0.6016285301999588